<a href="https://colab.research.google.com/github/lzziuhh/machine_learning/blob/master/yolov5_small_cots_training_improve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook based on below works, please vote!!!

https://www.kaggle.com/ammarnassanalhajali/barrier-reef-yolov5-training

since the limit of Kaggle GPU usage, I applied this notebook to colab

# Google drive connection

In [ ]:
import glob
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive

!ls /mydrive

Mounted at /content/gdrive
 bangali	    download	       JTdemo  'My Drive'
'Colab Notebooks'   download.gslides   kaggle


# Installing the [Kaggle API](https://github.com/Kaggle/kaggle-api) in Colab

In [ ]:
!pip install kaggle

# Authenticating with Kaggle using kaggle.json

Navigate to https://www.kaggle.com. Then go to the [Account tab of your user profile](https://www.kaggle.com/me/account) and select Create API Token. This will trigger the download of kaggle.json, a file containing your API credentials.

Then run the cell below to upload kaggle.json to your Colab runtime.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


# Using the Kaggle API

For a more complete list of what you can do with the API, visit https://github.com/Kaggle/kaggle-api.

## Listing competitions

In [ ]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         78           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        117           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        856           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        173           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1893           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      13725            True  
house-pr

## Downloading a dataset

In [ ]:
# unzip the download dataset
!mkdir /kaggle
!mkdir /kaggle/input
!mkdir /kaggle/input/tensorflow-great-barrier-reef


mkdir: cannot create directory ‘/kaggle’: File exists
mkdir: cannot create directory ‘/kaggle/input’: File exists
mkdir: cannot create directory ‘/kaggle/input/tensorflow-great-barrier-reef’: File exists


In [ ]:
%mkdir /kaggle/working
%cd /kaggle/working

/kaggle/working


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 10628, done.
remote: Total 10628 (delta 0), reused 0 (delta 0), pack-reused 10628
Receiving objects: 100% (10628/10628), 10.84 MiB | 14.86 MiB/s, done.
Resolving deltas: 100% (7335/7335), done.


In [ ]:
!cp /content/gdrive/MyDrive/kaggle/Yolo5/input/metrics.py /kaggle/working/yolov5/utils/metrics.py

In [ ]:
!cp /content/gdrive/MyDrive/kaggle/Yolo5/input/val.py /kaggle/working/yolov5/val.py

In [ ]:
!cp /content/gdrive/MyDrive/kaggle/Yolo5/input/__init__.py /kaggle/working/yolov5/utils/loggers/__init__.py

In [ ]:
# !kaggle competitions| download -c tensorflow-great-barrier-reef
#↑not works, only download about 40 pictures. 
# found dataset can be downloaded(thanks the help)
!kaggle datasets download -d yondraco/tensorflowgreatbarrierreef

100% 14.2G/14.2G [04:31<00:00, 57.4MB/s]
100% 14.2G/14.2G [04:31<00:00, 56.1MB/s]


In [ ]:
!unzip -q ./tensorflowgreatbarrierreef.zip -d /kaggle/input/tensorflow-great-barrier-reef

# Check GPU Type

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jan 23 13:17:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input director

In [ ]:
train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
train['pos'] = train.annotations != '[]'

In [ ]:
!mkdir -p ./yolo_data/fold1/images/val
!mkdir -p ./yolo_data/fold1/images/train

!mkdir -p ./yolo_data/fold1/labels/val
!mkdir -p ./yolo_data/fold1/labels/train

In [ ]:
fold = 1

annos = []
for i, x in train.iterrows():
    if x.video_id == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        if not x.pos: continue
        # val
    copyfile(f'../input/tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'./yolo_data/fold{fold}/images/{mode}/{x.image_id}.jpg')
    if not x.pos:
        continue
    r = ''
    anno = eval(x.annotations)
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'./yolo_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)

In [ ]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.5  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''

In [ ]:
data = '''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../yolo_data/fold1/  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['reef']  # class names


# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''

In [ ]:
with open('./yolov5/data/reef_f1_naive.yaml', 'w') as fp:
    fp.write(data)
with open('./yolov5/data/hyps/hyp.heavy.2.yaml', 'w') as fp:
    fp.write(hyps)

In [ ]:
%cd yolov5

/kaggle/working/yolov5


In [ ]:
!ls data/

Argoverse.yaml	      hyps		  scripts	 xView.yaml
coco128.yaml	      images		  SKU-110K.yaml
coco.yaml	      Objects365.yaml	  VisDrone.yaml
GlobalWheat2020.yaml  reef_f1_naive.yaml  VOC.yaml


In [ ]:
# Install W&B 
!pip install -q --upgrade wandb

# # Login 
# import wandb

# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient() 

# personal_key_for_api = user_secrets.get_secret("ke")
# ! wandb login $personal_key_for_api

# %pip install -q wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 3.9 MB/s 
     |████████████████████████████████| 142 kB 73.0 MB/s 
     |████████████████████████████████| 97 kB 9.0 MB/s 
     |████████████████████████████████| 180 kB 72.3 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#change parameter
metric recall0.3, map0.7 \
val 0.3-0,8-12


In [30]:
!python train.py --img 3072 \
                 --batch 4 \
                 --epochs 30 \
                 --data reef_f1_naive.yaml \
                 --weights yolov5s6.pt \
                 --project /content/gdrive/MyDrive/kaggle/Yolo5/kaggle-reef-5s \
                 --name 5s6_2176_0123 \
                 --hyp /content/hyp_evolve0119.yaml



wandb: Currently logged in as: lzziuhh (use `wandb login --relogin` to force relogin)
train: weights=yolov5s6.pt, cfg=, data=reef_f1_naive.yaml, hyp=/content/hyp_evolve0119.yaml, epochs=30, batch_size=4, imgsz=3072, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/gdrive/MyDrive/kaggle/Yolo5/kaggle-reef-5s, name=5s6_2176_0123, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-207-g8efe977 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: anchor_t=3.34433, anchors=2.4447, box=0.07117, cls=0.38718, cls_pw=0.52511, copy_paste=0.00299, degrees=0.0, fl_gamma=0.0,